In [ ]:
# Basic structure for neuron values + constants related to training & the activation f'n

ALPHA = 0.1  # leaky ReLU slope parameter
LEARNING_RATE = 0.01
BATCH_SIZE = 1

class Value:
    
    def __init__(self, val, grad=0.0):
        self.val = val
        self.grad = grad
    
    def __repr__(self):
        return "Value obj: " + str(self.val)

    def __add__(self, other):
        return Value(self.val + other.val, self.grad)

    def __mul__(self, other):
        return Value(self.val * other.val, self.grad)

    def __gt__(self, other):
        return self.val > other.val

    def reLU(self):
        return Value(max(0, self.val), self.grad)

    # leaky ReLU. slope parameter chosen arbitrarily
    def LreLU(self):
        if (self.val < 0):
            return Value(ALPHA * self.val, self.grad)
        return Value(self.val, self.grad)

In [ ]:
# ANN Structure

import random
import math

# 784 pixels for input, 10 possible outputs (0, 1,...,9), arbitrary hidden layers
MLP_LAYOUT = [784, 100, 100, 10]

class Neuron:

    def __init__(self, n, nout):
        #self.weights = [Value(random.uniform(-0.3,0.3)) for i in range(nout)]  # random uniform init
        std_dv = (2 / n) ** 0.5
        self.weights = [Value(random.gauss(0, std_dv)) for i in range(nout)]  # He Kaiming init
        self.val = Value(0.0)
        self.bias = Value(0.0)


class Layer:

    def __init__(self, n, nout):
        self.neurons = [Neuron(n, nout) for i in range(n)]       
        

class MLP:

    def __init__(self, MLP_LAYOUT):
        self.layers = [Layer(size, next_size) 
                       for size, next_size in zip(MLP_LAYOUT, MLP_LAYOUT[1:] + [0])]


    # currently O(n^3) T∆T
    def forward(self, inpt):
        # for every layer in mlp.layers[1:]
        # for every neuron2 in layer2
            # accum_val = neuron2bias
            # for every neuron1 in layer1
                # accum_val += neuronval * weight2
            # neuron2val = sigmoid(accum_val)
        
        # initialize neuron values to 0 before beginning
        for layer in self.layers:
            for neuron in layer.neurons:
                neuron.val.val = 0.0
    
        for i in range(len(inpt)):  # loading first layer of mlp
            self.layers[0].neurons[i].val.val = inpt[i]
    
        for layer1, layer2 in zip(self.layers[:-1], self.layers[1:]):
            for neuron2, n in zip(layer2.neurons, range(len(layer2.neurons))):
                neuron2.val += neuron2.bias;  # load with its bias
                for neuron1 in layer1.neurons:  # load with val*weight of each prev neuron
                    neuron2.val += neuron1.val * neuron1.weights[n]
                neuron2.val = neuron2.val.LreLU()  # commpression/activation f'n   
    
    # returns the result of the last forward pass
    def result(self):
        last_layer = self.layers[-1]
        x = 0
            
        for i in range(1, len(last_layer.neurons)):
            if (last_layer.neurons[i].val > last_layer.neurons[x].val):
                x = i
        return x

    # squared loss
    def loss(self, desired):
        last_layer = self.layers[-1]
        loss = 0.0
        for i in range(len(last_layer.neurons)):
            loss += (last_layer.neurons[i].val.val - desired[i])**2
        return loss

    # cross-entropy loss
    def lossCE(self, desired):
        neurons = self.layers[-1].neurons
        x = max([neurons[i].val.val for i in range(10)])
        denom = sum([math.exp(neurons[i].val.val - x) for i in range(10)])
        softmax = math.exp(neurons[desired.index(1)].val.val - x) / denom
        return -math.log(softmax)

    
    def backward(self, desired_output):
        # 1) zero out previous grads
        # 2) initialize grads of final layer considering cost f'n def'n
        # 3) for each layer in mlp_reverse except final layer
            # for each neuron in layer
                # for each weight from neuron
                    # gradw = (grad neuron of prev layer)*(sigmoid deriv)*val_neuron
                # if not in last layer...
                    # gradn = sum((grad neuron of prev layer)*(sigmoid deriv)*w)
                    # gradb = sum((grad neuron of prev layer)*(sigmoid deriv)*1)
        
        # 1)
        self.zero_grads()
    
        # 2)
        self.initgrads(desired_output)

        # 3)
        for layer2, layer1 in zip(reversed(self.layers[:-1]), reversed(self.layers[1:])):
            for neuron2 in layer2.neurons:
                # keep in mind; # of weights in any neuron of layer1 = # of neurons in layer2
                for neuron1, n in zip(layer1.neurons, range(len(layer1.neurons))):                    
                    if (neuron1.val.val < 0): x = ALPHA  # for leaky ReLU, this would be x = 1
                    else: x = 1  # x represents d(neuron1.val)/d(neuron1.val before ReLU) = dsig/da
              
                    neuron2.weights[n].grad = neuron1.val.grad * neuron2.val.val * x  # dC/dw = dC/dsig * dsig/da * da/dw
                    neuron2.val.grad += neuron1.val.grad * neuron2.weights[n].val * x  # dC/da = sum(dC/dw * dw/da)
                    neuron2.bias.grad += neuron1.val.grad * x  # dC/db = dC/dsig * dsig/da * da/db (last term is 1)
                # If we are in the last layer, no need to compute gradb or gradn
                # as they will never be used. Just unecessary
                # what we want: "while NOT in the last layer, then compute gradb & gradn"     


    def backward_v2(self):
        # copy-paste of part 3 of backward()
        for layer2, layer1 in zip(reversed(self.layers[:-1]), reversed(self.layers[1:])):
            for neuron2 in layer2.neurons:
                # keep in mind; # of weights in any neuron of layer1 = # of neurons in layer2
                for neuron1, n in zip(layer1.neurons, range(len(layer1.neurons))):                    
                    x = 1  # x represents d(neuron1.val)/d(neuron1.val before ReLU) = dsig/da
                    y = 1
                    if (neuron1.val.val < 0): x = ALPHA  #LReLU
                    if (neuron2.val.val < 0): y = ALPHA
              
                    neuron2.weights[n].grad = neuron1.val.grad * neuron2.val.val * x  # dC/dw = dC/dsig * dsig/da * da/dw
                    neuron2.val.grad += neuron1.val.grad * neuron2.weights[n].val * x  # dC/da = sum(dC/dw * dw/da)
                    neuron2.bias.grad += neuron1.val.grad * x * neuron2.weights[n].val * y
    
    
    # updates mlp using the gradients of each value
    def update(self):
        for layer in self.layers:
            for neuron in layer.neurons:
                # update bias
                neuron.bias.val -= LEARNING_RATE * neuron.bias.grad
                # update weights
                for weight in neuron.weights:
                    weight.val -= LEARNING_RATE * weight.grad
                    

    # initializes first gradient layer based on softmax and categorical CE loss
    def initgrads(self, desired_output):
        val_gradient = 0.0
        
        neurons = self.layers[-1].neurons
        x = max([neurons[j].val.val for j in range(10)])  # to avoid math overflow error, shifts exp values towards 0
        for i in range(len(desired_output)):
            denom = sum([math.exp(neurons[j].val.val - x) for j in range(10)])
            if desired_output[i] == 0:  # for non-target logits, grad is softmax of this logit
                val_gradient = math.exp(neurons[i].val.val - x) / denom
            else:  # for target logit, grad is [softmax of this logit - 1]
                val_gradient = (math.exp(neurons[i].val.val - x) / denom) - 1
            neurons[i].val.grad += val_gradient
            
            if neurons[i].val.val < 0:
                neurons[i].bias.grad += val_gradient * ALPHA
            else: neurons[i].bias.grad += val_gradient


    def divide_first_layer_grads(self):
        for neuron in self.layers[-1].neurons:
            neuron.val.grad /= BATCH_SIZE
            neuron.bias.grad /= BATCH_SIZE
    

    def zero_grads(self):
        for layer in self.layers:
            for neuron in layer.neurons:
                neuron.val.grad = 0.0
                neuron.bias.grad = 0.0
                for weight in neuron.weights:
                    weight.grad = 0.0


In [ ]:
# Data driver

import keras
from keras.datasets import mnist
(train_x, train_y), (test_x, test_y) = mnist.load_data()

class Data():

    # i=1 for testing, i=0 (default) for training
    def __init__(self, num_exercises, i=0):
        self.inpt = []
        self.outpt = []
        if (i == 0):
            self.load_data(train_x, train_y, num_exercises)
        else: 
            self.load_data(test_x, test_y, num_exercises)
    
    def load_data(self, t_x, t_y, n):
        t = []
        d = []
        for i in range(n):  # loads n examples
            self.inpt.append(t_x[i].flatten())
            self.outpt.append(t_y[i].flatten())
        self.normalize_inpt()
        self.normalize_outpt()
        return (t, d)

    # maps 0-255 pixel values to values b/t 0 and 1
    def normalize_inpt(self):
        for i in range(len(self.inpt)):
            self.inpt[i] = self.inpt[i] / 255.0

    # casts desired output into list with a 1 at the index equal to 
    # desired output (i.e. if desired output is 6, gives 
    # desired[i] = [0 0 0 0 0 0 1 0 0 0])
    def normalize_outpt(self):
        for i in range(len(self.outpt)):
            x = self.outpt[i].item()
            self.outpt[i] = []
            for j in range(10):  # 10 possible digits, one index for each
                if (x == j): self.outpt[i].append(1)
                else: self.outpt[i].append(0)

# loading training data
train = Data(60000)
print(len(train.inpt[0]), train.outpt[0])

# loading testing data
test = Data(10000, 1)
print(len(test.inpt[0]), test.outpt[0])

In [ ]:
# Saving all ANN info to external csv for debugging and general analysis

import csv
EMPTY = "          "
with open('0_all_biases.csv', 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow(['B0', 'B1', 'B2', 'B3', '', 'BG0', 'BG1', 'BG2', 'BG3'])
    
    for i in range(max([len(mlp.layers[i].neurons) for i in range(len(mlp.layers))])):  # 'for neuron in layer with most neurons'
        row = [0.0 for j in range(2 * len(mlp.layers) + 1)]
        for j in range(len(mlp.layers)):  # biases
            try: row[j] = mlp.layers[j].neurons[i].bias.val
            except: row[j] = EMPTY
        row[len(mlp.layers)] = EMPTY  # space column for readability
        for j in range(len(mlp.layers) + 1, 2*len(mlp.layers) + 1):  # bias gradients
            try: row[j] = mlp.layers[j - len(mlp.layers) - 1].neurons[i].bias.grad
            except: row[j] = EMPTY
        writer.writerow(row)

with open('0_all_neurons.csv', 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow(['V0', 'V1', 'V2', 'V3', '', 'VG0', 'VG1', 'VG2', 'VG3'])
    
    for i in range(max([len(mlp.layers[i].neurons) for i in range(len(mlp.layers))])):  # 'for neuron in layer with most neurons'
        row = [0.0 for j in range(2 * len(mlp.layers) + 1)]
        for j in range(len(mlp.layers)):  # values
            try: row[j] = mlp.layers[j].neurons[i].val.val
            except: row[j] = EMPTY
        row[len(mlp.layers)] = EMPTY  # space column for readability
        for j in range(len(mlp.layers) + 1, 2*len(mlp.layers) + 1):  # value gradients
            try: row[j] = mlp.layers[j - len(mlp.layers) - 1].neurons[i].val.grad
            except: row[j] = EMPTY
        writer.writerow(row)

with open('0_all_weights.csv', 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow(['W0', 'W1', 'W2', 'W3', '', 'WG0', 'WG1', 'WG2', 'WG3'])

    for i in range(max([len(mlp.layers[x].neurons) for x in range(len(mlp.layers))])):  # 'for neuron in layer with most neurons'
        row = [0.0 for j in range(2 * len(mlp.layers) + 1)]
        for w in range(max([len(mlp.layers[x].neurons[0].weights) for x in range(len(mlp.layers))])):  # 'for weight in layer with most weights'
            for j in range(len(mlp.layers)):  # weights
                try: row[j] = mlp.layers[j].neurons[i].weights[w].val
                except: row[j] = EMPTY
            row[len(mlp.layers)] = EMPTY  # space column for readability
            for j in range(len(mlp.layers) + 1, 2*len(mlp.layers) + 1):  # weight gradients
                try: row[j] = mlp.layers[j - len(mlp.layers) - 1].neurons[i].weights[w].grad
                except: row[j] = EMPTY
            writer.writerow(row)
        writer.writerow([EMPTY for i in range(2 * len(mlp.layers) + 1)])

In [ ]:
# Initializing a new ANN instance
mlp = MLP(MLP_LAYOUT)

In [ ]:
# Overfitting to one training example

TRAINING_EXAMPLE = 19
print(train.outpt[TRAINING_EXAMPLE])

for i in range(1000):
    mlp.zero_grads()
    mlp.forward(train.inpt[TRAINING_EXAMPLE])
    print(mlp.lossCE(train.outpt[TRAINING_EXAMPLE]))
    mlp.initgrads(train.outpt[TRAINING_EXAMPLE])
    mlp.backward_v2()
    mlp.update()

In [ ]:
# Testing

test = Data(10000, 1)
correct = 0.0
total = 0.0
for i in range(10000):
    total += 1
    mlp.forward(test.inpt[i])
    if (test.outpt[i].index(1) == mlp.result()): 
        correct += 1
        print(i, "            exp: ", test.outpt[i].index(1), "act: ", mlp.result(), "         ", correct / total, "             !")
    else: print(i, "            exp: ", test.outpt[i].index(1), "act: ", mlp.result(), "         ", correct / total)


In [ ]:
# Training

counter = 0
l = [0.0 for i in range(BATCH_SIZE)]
for i in range(52499, 60000):
    mlp.forward(train.inpt[i])
    mlp.initgrads(train.outpt[i])
    l[counter] = mlp.lossCE(train.outpt[i])
    counter += 1
    
    if (counter == BATCH_SIZE):
        av_loss = sum(l) / BATCH_SIZE
        print(i, ". ", av_loss)
        counter = 0
        mlp.divide_first_layer_grads()
        mlp.backward_v2()
        mlp.update()
        mlp.zero_grads()

In [ ]:
# Training to overfit on a particular set of training examples

for i in range(30):
    for j in range(160):
        mlp.forward(train.inpt[i])
        mlp.initgrads(train.outpt[i])
        l[counter] = mlp.lossCE(train.outpt[i])
        counter += 1
        
        if (counter == BATCH_SIZE):
            av_loss = sum(l) / BATCH_SIZE
            print(av_loss)
            counter = 0
            mlp.divide_first_layer_grads()
            mlp.backward_v2()
            mlp.update()
            mlp.zero_grads()